In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# Gốc project 
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "dataset"

SUM_EXT_PATH   = DATA_DIR / "summarize_data.csv"   
DATA2_PATH     = DATA_DIR / "data2.xlsx"          
COMBINED_PATH  = DATA_DIR / "summarize_data_combined.csv"

print("SUM_EXT_PATH  :", SUM_EXT_PATH,  "->", SUM_EXT_PATH.exists())
print("DATA2_PATH    :", DATA2_PATH,    "->", DATA2_PATH.exists())
print("COMBINED_PATH :", COMBINED_PATH)


SUM_EXT_PATH  : d:\do-an-tot-nghiep\dataset\summarize_data.csv -> True
DATA2_PATH    : d:\do-an-tot-nghiep\dataset\data2.xlsx -> True
COMBINED_PATH : d:\do-an-tot-nghiep\dataset\summarize_data_combined.csv


In [3]:
df_ext = pd.read_csv(SUM_EXT_PATH)
print("summarize_data.csv:", df_ext.shape)
print("Cột extractive:", df_ext.columns.tolist())


df2 = pd.read_excel(DATA2_PATH)
print("data2.xlsx:", df2.shape)
print("Cột data2:", df2.columns.tolist())


summarize_data.csv: (11353, 3)
Cột extractive: ['input_text', 'target_text', 'label']
data2.xlsx: (42833, 5)
Cột data2: ['input', 'output', 'sentences', 'input_sentences', 'ratio']


Lọc mẫu hợp chuẩn 

In [4]:
# Tính độ dài input/output theo số từ
df2["in_words"]  = df2["input"].astype(str).str.split().str.len()
df2["out_words"] = df2["output"].astype(str).str.split().str.len()

print("Mô tả độ dài input/output (words):")
print(df2[["in_words", "out_words"]].describe(percentiles=[0.5, 0.9, 0.95, 0.99]))

# Điều kiện "summary-like"
mask_len = df2["out_words"] <= 220              # target không quá dài
mask_ratio_len = df2["out_words"] <= 0.8 * df2["in_words"]   # ngắn hơn input rõ
mask_flag = df2["ratio"] == 0                   # ưu tiên các mẫu ratio=0

mask_good = mask_len & mask_ratio_len & mask_flag

df_abs = df2[mask_good].copy()
print("\nSố mẫu thỏa điều kiện:", len(df_abs))

# Nếu còn > 4000 mẫu thì random chọn khoảng 4000
N = 4000
if len(df_abs) > N:
    df_abs = df_abs.sample(n=N, random_state=42).reset_index(drop=True)
else:
    df_abs = df_abs.sample(frac=1.0, random_state=42).reset_index(drop=True)

print("Số mẫu abstractive-like dùng để gộp:", len(df_abs))


Mô tả độ dài input/output (words):
           in_words     out_words
count  42833.000000  42833.000000
mean     633.706932    496.386478
std      337.532991    315.551602
min       36.000000     40.000000
50%      565.000000    416.000000
90%     1059.800000    922.000000
95%     1233.000000   1086.000000
99%     1710.680000   1484.000000
max     5043.000000   3899.000000

Số mẫu thỏa điều kiện: 3986
Số mẫu abstractive-like dùng để gộp: 3986


Chuẩn hoá schema & gộp

In [5]:
# Chuẩn schema extractive
need_cols = ["input_text", "target_text"]
missing_ext = [c for c in need_cols if c not in df_ext.columns]
if missing_ext:
    raise ValueError(f"summarize_data.csv thiếu cột: {missing_ext}")

df_ext_small = df_ext[need_cols].dropna()
print("Extractive sau lọc:", df_ext_small.shape)

# Chuẩn schema abstractive-from-data2
df_abs_small = pd.DataFrame({
    "input_text": df_abs["input"].astype(str).str.strip(),
    "target_text": df_abs["output"].astype(str).str.strip(),
}).dropna()

print("Abstractive (từ data2) sau lọc:", df_abs_small.shape)

# Gộp & shuffle
combined_df = pd.concat([df_ext_small, df_abs_small], ignore_index=True)
combined_df = combined_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

print("\nTổng mẫu sau gộp:", combined_df.shape)
print(combined_df.head())


Extractive sau lọc: (11353, 2)
Abstractive (từ data2) sau lọc: (3986, 2)

Tổng mẫu sau gộp: (15339, 2)
                                          input_text  \
0  Y án tử hình kẻ sát hại cô gái, bỏ xác vào val...   
1  Carlsen gieo thêm sầu cho Vua cờ Gukesh. Carls...   
2  Tạm đình chỉ công tác hiệu trưởng vì để thực p...   
3  Tổng Bí thư thăm lớp học mang tên Chủ tịch Hồ ...   
4  Công Phượng chưa hẹn ngày trở lại, Đồng Nai có...   

                                         target_text  
0  Thanh Phương Tòa phúc thẩm TAND tối cao tại TP...  
1  Carlsen thắng cả 4 ván gặp Gukesh và kỳ thủ số...  
2  Xuân Ngọc UBND phường Xuân Hương - Đà Lạt (Lâm...  
3  Tại đây, các học sinh đã trình bày những tiết ...  
4  Với Đồng Nai, nếu giành 3 điểm, họ tiếp tục du...  


Lưu file combined mới

In [6]:
combined_df.to_csv(COMBINED_PATH, index=False, encoding="utf-8-sig")

print(
    f"Đã gộp {len(df_ext_small)} mẫu extractive + "
    f"{len(df_abs_small)} mẫu abstractive "
    f"-> Tổng {len(combined_df)} mẫu."
)
print("Đã lưu dataset kết hợp mới:", COMBINED_PATH)


Đã gộp 11353 mẫu extractive + 3986 mẫu abstractive -> Tổng 15339 mẫu.
Đã lưu dataset kết hợp mới: d:\do-an-tot-nghiep\dataset\summarize_data_combined.csv


In [7]:
# Tính độ dài input và target theo số từ
combined_df["input_len"] = combined_df["input_text"].astype(str).str.split().str.len()
combined_df["target_len"] = combined_df["target_text"].astype(str).str.split().str.len()

# Tính compression ratio
combined_df["compression_ratio"] = combined_df["target_len"] / combined_df["input_len"]

# Thống kê
print("=== Thống kê độ dài và compression ratio ===")
print(f"Độ dài input trung bình: {combined_df['input_len'].mean():.2f} từ")
print(f"Độ dài target trung bình: {combined_df['target_len'].mean():.2f} từ")
print(f"Compression ratio trung bình: {combined_df['compression_ratio'].mean():.2%}")

print("\nMô tả chi tiết:")
print(combined_df[["input_len", "target_len", "compression_ratio"]].describe())

=== Thống kê độ dài và compression ratio ===
Độ dài input trung bình: 600.40 từ
Độ dài target trung bình: 184.82 từ
Compression ratio trung bình: 37.29%

Mô tả chi tiết:
          input_len    target_len  compression_ratio
count  15339.000000  15339.000000       15339.000000
mean     600.395267    184.822087           0.372858
std      292.233821     32.293311           0.169216
min       54.000000     45.000000           0.042115
25%      395.000000    169.000000           0.249414
50%      536.000000    196.000000           0.337248
75%      747.000000    209.000000           0.454057
max     1977.000000    220.000000           0.973214
